## Análise de Decisão

### O problema do Preço Certo.

Para um pensador bayesiano, esse cenário sugere várias perguntas:
- Antes de ver os prêmios, que crenças priori o competidor deve ter sobre o preço da vitrine?
- Depois de ver os prêmios, como o competidor deve atualizar essas crenças?
- **Com base na distribuição posteriori, o que o concorrente deve oferecer?**

### A Priori

Para escolher uma **distribuição priori** dos preços, podemos tirar proveito dos dados dos **episódios anteriores**.

O valor mais comum para as duas mostras é de cerca de US$ 28.000;

Distribuições baseadas em dados reais, mas suavizadas pela estimativa de densidade de kernel Gaussiana (KDE).

### Função de densidade de probabilidade

- **PDF** é a versão **contínua** de um **PMF**;

- As PDFs geralmente são escritas como funções. Para um determinado valor de x, essa função calcula uma densidade de probabilidade;

- Densidade é semelhante a uma massa de probabilidade, mas não é uma probabilidade;

- A integral da densidade resulta em uma probabilidade;

### Representando PDFs

-  `thinkbayes.py` fornece uma **Pdf** do tipo abstrata.
    - Density;
    - MakePmf;

- Implementação de uma distribuição gaussiana;

Podemos usar uma amostra para estimar o PDF de toda a população.

A **estimativa de densidade de kernel (KDE)** é um **algoritmo** que recebe uma amostra e **encontra uma PDF** que se **ajusta aos dados**. 

In [2]:
from price import ReadData, Player
import scipy
import numpy
import thinkbayes
prices = numpy.array([i[0] for i in ReadData(filename='showcases.2012.csv')])
bids = numpy.array([i[2] for i in ReadData(filename='showcases.2012.csv')])
diffs = numpy.array([i[4] for i in ReadData(filename='showcases.2012.csv')])
player = Player(prices, bids, diffs)

In [3]:
pmf = player.PmfPrice()

### Modelando os competidores

Para atualizar essas priori, precisamos responder estas perguntas:

- Quais dados devemos considerar e como quantificá-los?
- Podemos calcular uma função de probabilidade; isto é, para cada valor hipotético do preço , podemos calcular a probabilidade condicional dos dados?

Modelou o concorrente como um instrumento de estimativa de preço com características de erro conhecidas;
- Se o preço real é preço, qual é a verossimilhança de que a estimativa do concorrente seja adivinhada?
- Qual é a verossimilhança do competidor ser desligado pelo erro da estimativa?

Podemos usar essa distribuição para estimar a confiabilidade das suposições dos competidores.
- Faz algumas suposições:
    -  Distribuição do erro seja gaussiana com média 0 e a mesma variância que diff.

### Verossimilhança

In [1]:
def Likelihood(self, data, hypo):
    price = hypo
    guess = data

    error = price - guess
    like = self.player.ErrorDensity(error)

    return like

- Escreve a função de verossimilhança;
- **hypo** é o preço hipotético da vitrine;
- **data** são o melhor palpite do competidor
- **erro** diferença;
- **like** é a verossimilhança dos dados; 

### Atualização

O **Player** fornece um método que aceita o **palpite do competidor** e calcula a distribuição posteriori;

In [5]:
def MakeBeliefs(self, guess):
    pmf = self.PmfPrice()
    self.prior = Price(pmf, self)
    self.posterior = self.prior.Copy()
    self.posterior.Update(guess)

Combinando duas fontes de informação, dados históricos sobre mostras passadas e suposições sobre os prêmios que vê.

Estamos tratando os **dados históricos como priori** e atualizando-os com base em suas suposições, mas poderíamos usar sua **suposição como priori** e atualizá-los com base em dados históricos.


### Lance ideal

Podemos usar a distribuição posteriori para máximizar as chances (**Retorno Esperado**);

`def ExpectedGain` percorre os valores na parte posterior e calcula o ganho de cada lance;

Dado um palpite e um oponente, o OptimalBid calcula a distribuição posteriori;

Calcula os ganhos esperados para uma série de lances e retorna o **lance ideal** e o **ganho esperado**;

### Discussão

- Uma das características da **estimativa bayesiana** é que o resultado vem na forma de uma **distribuição posteriori**, diferentes da **estimativa clássica** que retorna um **valor único** ou um **intervalo de confiança**;


- Esses resumos (da distribuição posteriori) podem ser úteis, mas se é tudo o que você precisa, provavelmente não precisará de métodos bayesianos;

- Os métodos bayesianos são mais úteis quando você pode **levar** a distribuição posteriori para a **próxima etapa** da análise para executar algum tipo de análise de decisão;